## Postgres on GCP with Vector Extension   

In [ ]:
!pip3 install google-cloud-aiplatform --upgrade

In [1]:
##############################################################
## Connection Postgres & Queries
##############################################################
# code based from: https://github.com/GoogleCloudPlatform/cloud-sql-python-connector.git

from google.cloud.sql.connector import Connector # pip install "cloud-sql-python-connector[pg8000]"
import sqlalchemy # pip install sqlalchemy
from template_specification_gcp_postgres import project_id, region, instance_name, current_user, DB_USER, DB_PASS, DB_NAME

# to move out of the document before uploading to Github
project_id = "ucl-engineering-invoice"
region = "us-central1"
instance_name = "quickstart-user"
current_user = ['alessandra.eli.cerutti@gmail.com']

DB_USER = "quickstart-user" 
DB_PASS = "quick-ale"
DB_NAME = "text-extraction"

# Define Connection name
INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" 
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")

# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

Your instance connection name is: ucl-engineering-invoice:us-central1:quickstart-user


In [11]:
# create table to add the extracted data from the documents thanks to the functions created in the text extraction.py
# table should have three columns: unique ID, file name, extracted_text

# to move out of the document before uploading to Github
project_id = "ucl-engineering-invoice"
region = "us-central1"
instance_name = "quickstart-user"
current_user = ['alessandra.eli.cerutti@gmail.com']

DB_USER = "postgres" 
DB_PASS = "quick-ale"
DB_NAME = "text-extraction"

#################################################################
# CREATE a new table
#################################################################

def create_new_table_postgres():
    # Use the getconn function to connect to the database
    con = getconn()

    # Create a cursor from the connection
    cursor = con.cursor()

    # SQL Query to create the new table
    # data structured based on the spark output
    create_table_query = """
    CREATE TABLE invoices (
        unique_id SERIAL PRIMARY KEY,
        file_name TEXT,
        extracted_text TEXT
    );
    """

    try:
        # Execute the create table query
        cursor.execute(create_table_query)

        # Commit the changes to the database
        con.commit()
        print("New table 'invoices' created successfully.")
    except Exception as e:
        # If an error occurs, print it and rollback any changes
        print(f"An error occurred: {e}")
        con.rollback()
    finally:
        # Close the cursor and connection
        cursor.close()
        con.close()

create_new_table_postgres()


New table 'invoices' created successfully.


In [19]:
##### Test text extraction from Bucket 

import os
import tempfile
from google.cloud import storage
from pdf2image import convert_from_path
from pytesseract import image_to_string

# Initialize GCS client
storage_client = storage.Client()

def download_blob_to_temp(bucket_name, source_blob_name):
    """Downloads a blob from the bucket to a temporary file."""
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    _, temp_local_filename = tempfile.mkstemp()
    blob.download_to_filename(temp_local_filename)
    return temp_local_filename

def convert_pdf_to_img(pdf_file):
    """Converts a PDF file to a list of images."""
    return convert_from_path(pdf_file)

def convert_image_to_text(file):
    """Converts an image file to text."""
    text = image_to_string(file)
    return text

def get_text_from_any_pdf(pdf_file):
    """Converts any given PDF file to text by first converting PDF pages to images."""
    images = convert_pdf_to_img(pdf_file)
    final_text = ""
    for pg, img in enumerate(images):
        final_text += convert_image_to_text(img)
    return final_text

def process_pdfs_in_bucket(bucket_name):
    """Processes all PDF files in the specified GCS bucket."""
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs()

    for blob in blobs:
        if blob.name.endswith('.pdf'):
            print(f"Processing {blob.name}...")
            temp_pdf_path = download_blob_to_temp(bucket_name, blob.name)
            try:
                extracted_text = get_text_from_any_pdf(temp_pdf_path)
                print(f"Extracted text from {blob.name}: {extracted_text[:100]}...")  # Printing first 100 characters for brevity
            finally:
                os.remove(temp_pdf_path)  # Clean up temporary file

# Replace 'your-bucket-name' with your GCS bucket name
process_pdfs_in_bucket('invoice_scan')

Processing invoice_1.pdf...
Extracted text from invoice_1.pdf: Restaurant du
CLUB NAUTIQUE MORGIFN

PI. de la Navigation 1

1110 Morges
Facture 983 167
Numéro de c...
Processing invoice_10.pdf...
Extracted text from invoice_10.pdf: Verein
Berufliche Integration CHANCE Z!
Niklaus-Thus-Platz 19

4800 Zofingen
Rechnung 983 986
Kunden...
Processing invoice_2.pdf...
Extracted text from invoice_2.pdf: HOTEL SALUVER AG
Via Maistra 128
7505 Celerina
Rechnung 983 898
Kundennummer 3'277 D1
Sachebearbeite...
Processing invoice_3.pdf...
Extracted text from invoice_3.pdf: Fattura

983 360

Numero cliente 7'726

Collaboratore

Isabella

Indirizzo di fornitura:

FD - Pagam...
Processing invoice_5.pdf...
Extracted text from invoice_5.pdf: Fattura

983 201

Numero cliente 11'004

Collaboratore

Rosanna

Indirizzo di fornitura:

Bollettino...
Processing invoice_6.pdf...
Extracted text from invoice_6.pdf: Firma

GVS Landi AG
Vinothek
Gennersbrunnerstrasse 61
8207 Schaffhausen
Rechnung 981 341
Kundennumme

In [15]:
def insert_extracted_text_into_db(file_name, extracted_text):
    """Inserts the file name and extracted text into a PostgreSQL database."""
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    cur = conn.cursor()
    cur.execute("INSERT INTO invoices (file_name, extracted_text) VALUES (%s, %s)", (file_name, extracted_text))
    conn.commit()
    cur.close()
    conn.close()

def process_pdfs_in_bucket(bucket_name):
    """Processes all PDF files in the specified GCS bucket and saves the extracted text in a PostgreSQL table."""
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs()

    for blob in blobs:
        if blob.name.endswith('.pdf'):
            print(f"Processing {blob.name}...")
            temp_pdf_path = download_blob_to_temp(bucket_name, blob.name)
            try:
                extracted_text = get_text_from_any_pdf(temp_pdf_path)
                # Insert into the database
                insert_extracted_text_into_db(blob.name, extracted_text)
                print(f"Inserted extracted text from {blob.name} into the database.")
            finally:
                os.remove(temp_pdf_path)  # Clean up the temporary file

# Apply extracting text to bucket
process_pdfs_in_bucket('invoice_scan')

Processing invoice_1.pdf...
Inserted extracted text from invoice_1.pdf into the database.
Processing invoice_10.pdf...
Inserted extracted text from invoice_10.pdf into the database.
Processing invoice_2.pdf...
Inserted extracted text from invoice_2.pdf into the database.
Processing invoice_3.pdf...
Inserted extracted text from invoice_3.pdf into the database.
Processing invoice_5.pdf...
Inserted extracted text from invoice_5.pdf into the database.
Processing invoice_6.pdf...
Inserted extracted text from invoice_6.pdf into the database.
Processing invoice_7.pdf...
Inserted extracted text from invoice_7.pdf into the database.
Processing invoice_8.pdf...
Inserted extracted text from invoice_8.pdf into the database.


In [17]:
#################################################################
##### Checking Table Invoices 
#################################################################
def list_invoices():
    # Use the getconn function to connect to the database
    conn = getconn()
    
    # Create a cursor from the connection
    cursor = conn.cursor()

    # SQL Query to select the first 5 rows from the 'weather_data' table
    query = """
        SELECT *
        FROM invoices;
    """
    
    try:
        # Execute the query
        cursor.execute(query)

        # Fetch all the results
        rows = cursor.fetchall()

        # Print the rows in a more readable format
        for row in rows:
            print(row)  # Or use any other method to format the output

    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

# Call the function to print out the first 5 entries of the invoices table
list_invoices()

[1, 'invoice_1.pdf', "Restaurant du\nCLUB NAUTIQUE MORGIFN\n\nPI. de la Navigation 1\n\n1110 Morges\nFacture 983 167\nNuméro de client 2'262 F3A\nCollaborateur Rosanna\n—SC“C:sSCSC‘SS Novazzano 21.02.2024 Page 1/2\nAdresse de fourniture: Restaurant du, CLUB NAUTIQUE MORGIEN, B. de la Navigation 1, 1110 Morges\n\nFD (P)-posta1\n\nNo. article / désignation Quantité\nE201G Café Extra-Milano en grains 24.00 pcs. 23.00 552.00 23\n1000g\nNo lot 24LT16FE1, 24.00 pcs.\n16.08.2025\nTotal intermédiaire 552.00\nTVA 2.6% net (Code 23) de 552.00 14.35\n\nTotal CHF\n\nConditions de paiement: 30 jours net.\n\n"]
[2, 'invoice_10.pdf', "Verein\nBerufliche Integration CHANCE Z!\nNiklaus-Thus-Platz 19\n\n4800 Zofingen\nRechnung 983 986\nKundennummer 11'584 Dgs\nSachebearbeiter Rosanna\nNovazzano 11.03.2024 Seite 1/2\nLiefersadresse: Verein, Berufliche Integration CHANCE Z!, Niklaus-Thus-Platz 19, 4800 Zofingen\nFD - Posta 1 (Zahlung: Bankueberweisung)\n\nSPEDIRE A: Wunschladen.ch, Herr Peter Frey,Hintere

['ucl-engineering-invoice:us-central1:quickstart-user']: An error occurred while performing refresh. Scheduling another refresh attempt immediately
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/google/cloud/sql/connector/instance.py", line 298, in _refresh_task
    refresh_data = await refresh_task
                   ^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/google/cloud/sql/connector/instance.py", line 233, in _perform_refresh
    metadata = await metadata_task
               ^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/google/cloud/sql/connector/client.py", line 124, in _get_metadata
    resp = await self._client.get(url, headers=headers, raise_for_status=True)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/aiohttp/client.py", line 605, in _request
    await resp.start(conn)
  File "/opt/anaconda3/lib/pyth

## Alter the table `invoices` to add the vectorised text

In [2]:
# to move out of the document before uploading to Github
project_id = "ucl-engineering-invoice"
region = "us-central1"
instance_name = "quickstart-user"
current_user = ['alessandra.eli.cerutti@gmail.com']

DB_USER = "postgres" 
DB_PASS = "quick-ale"
DB_NAME = "text-extraction"

def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn


In [24]:


def new_col_for_vectors_invoices():
    # Use the getconn function to connect to the database
    conn = getconn()

    # Create a cursor from the connection
    cursor = conn.cursor()

    # SQL Query to add a new column for text vectors to the 'invoices' table
    query = "ALTER TABLE invoices ADD COLUMN text_vectors float8[];"

    try:
        # Execute the query
        cursor.execute(query)
        # Commit the changes
        conn.commit()
    except Exception as e:
        # Print the exception if any occurs
        print("An error occurred:", e)
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

new_col_for_vectors_invoices()


## Vectorising the sentences

Connect to postgres SQL

In [2]:
import psycopg2
from google.cloud.sql.connector import Connector # pip install "cloud-sql-python-connector[pg8000]"
import sqlalchemy # pip install sqlalchemy
from template_specification_gcp_postgres import project_id, region, instance_name, current_user, DB_USER, DB_PASS, DB_NAME

IP_ADRESS = '35.232.156.133'
DB_USER = "postgres" 
DB_PASS = "quick-ale"
DB_NAME = "text-extraction"

# Define the instance connection name
INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" 
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")

# Initialize Connector object
connector = Connector()

# Function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# Create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)



Your instance connection name is: ucl-engineering-invoice:us-central1:quickstart-user


In [16]:
import pandas as pd
from sqlalchemy import text

# Function to fetch data from the database
def fetch_data():
    with pool.connect() as conn:
        # Query to select relevant columns
        query = text("SELECT unique_id, extracted_text FROM invoices")
        result = conn.execute(query)
        # Create a DataFrame from the query results
        df = pd.DataFrame(result.fetchall(), columns=["unique_id", "extracted_text"])
        return df

# Fetch the data
df = fetch_data()

# Display the first few rows of the DataFrame to confirm successful data retrieval
print(df.head())

   unique_id                                     extracted_text
0          1  Restaurant du\nCLUB NAUTIQUE MORGIFN\n\nPI. de...
1          2  Verein\nBerufliche Integration CHANCE Z!\nNikl...
2          3  HOTEL SALUVER AG\nVia Maistra 128\n7505 Celeri...
3          4  Fattura\n\n983 360\n\nNumero cliente 7'726\n\n...
4          5  Fattura\n\n983 201\n\nNumero cliente 11'004\n\...


Split Data in manageable chunks

In [19]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 9.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.6/250.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
  Attempting uninstall: jsonpatch
    Found existing installation: jsonpatch 1.32
    Uninstalling jsonpatch-1.32:
      Successfully uninstalled jsonpatch-1.32


In [21]:
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n"],
    chunk_size=500,
    chunk_overlap=80,
    length_function=len
)

# Split the text into chunks
chunked = []
for index, row in df.iterrows():
    invoice_id = row["unique_id"]
    text = row["extracted_text"]
    splits = text_splitter.create_documents([text])
    for s in splits:
        chunked.append({"invoice_id": invoice_id, "content": s.page_content})

In [31]:
df.head()

,unique_id,extracted_text
0,1,Restaurant du\nCLUB NAUTIQUE MORGIFN\n\nPI. de...
1,2,Verein\nBerufliche Integration CHANCE Z!\nNikl...
2,3,HOTEL SALUVER AG\nVia Maistra 128\n7505 Celeri...
3,4,Fattura\n\n983 360\n\nNumero cliente 7'726\n\n...
4,5,Fattura\n\n983 201\n\nNumero cliente 11'004\n\...


In [32]:
chunked

[{'invoice_id': 1,
  'content': "Restaurant du\nCLUB NAUTIQUE MORGIFN\n\nPI. de la Navigation 1\n\n1110 Morges\nFacture 983 167\nNuméro de client 2'262 F3A\nCollaborateur Rosanna\n—SC“C:sSCSC‘SS Novazzano 21.02.2024 Page 1/2\nAdresse de fourniture: Restaurant du, CLUB NAUTIQUE MORGIEN, B. de la Navigation 1, 1110 Morges\n\nFD (P)-posta1\n\nNo. article / désignation Quantité\nE201G Café Extra-Milano en grains 24.00 pcs. 23.00 552.00 23\n1000g\nNo lot 24LT16FE1, 24.00 pcs.\n16.08.2025\nTotal intermédiaire 552.00\nTVA 2.6% net (Code 23) de 552.00 14"},
 {'invoice_id': 1,
  'content': '.\n16.08.2025\nTotal intermédiaire 552.00\nTVA 2.6% net (Code 23) de 552.00 14.35\n\nTotal CHF\n\nConditions de paiement: 30 jours net.'},
 {'invoice_id': 2,
  'content': "Verein\nBerufliche Integration CHANCE Z!\nNiklaus-Thus-Platz 19\n\n4800 Zofingen\nRechnung 983 986\nKundennummer 11'584 Dgs\nSachebearbeiter Rosanna\nNovazzano 11.03.2024 Seite 1/2\nLiefersadresse: Verein, Berufliche Integration CHANCE Z!,

Setting up Google Cloud Vertex AI for embeddings

In [45]:
from google.cloud import aiplatform
from langchain.embeddings import VertexAIEmbeddings

# Initialize AI Platform
project_id = "ucl-engineering-invoice"
region = "us-central1"
aiplatform.init(project=project_id, location=region)
embeddings_service = VertexAIEmbeddings()

Model_name will become a required arg for VertexAIEmbeddings starting from Feb-01-2024. Currently the default is set to textembedding-gecko@001


In [46]:
from google.cloud import aiplatform_v1beta1 as aiplatform

# Initialize the Vertex AI client
project_id = "your_project_id"
region = "us-central1"
api_endpoint = f"{region}-aiplatform.googleapis.com"

client_options = {"api_endpoint": api_endpoint}
embeddings_client = aiplatform.services.prediction_service.PredictionServiceClient(client_options=client_options)

# Specify the parent resource
parent = f"projects/{project_id}/locations/{region}"



In [47]:
print(embeddings_client)

In [49]:
def handle_vertex_ai_response(request_content):
    embedding_requests = []
    for content in request_content:
        embedding_request = aiplatform_gapic.EmbeddingRequest(content=content)
        embedding_requests.append(embedding_request)

    # Batch embedding call
    request = aiplatform_gapic.BatchCreateDocumentEmbeddingsRequest(
        parent=parent,
        requests=embedding_requests,
    )

    response = embeddings_client.batch_create_document_embeddings(request=request)

    # Assuming each response has embeddings that we can iterate over
    embeddings = []
    for embedding_response in response.responses:
        # If the response has an error, handle it appropriately
        if embedding_response.error.message:
            print(f"Error processing document: {embedding_response.error.message}")
            embeddings.append(None)
        else:
            # Extract the embedding from the response
            document_embedding = embedding_response.embeddings
            embeddings.append(document_embedding)

    return embeddings


In [50]:
from google.cloud import aiplatform

# Initialize the client for Vertex AI
client_options = {
    "api_endpoint": "us-central1-aiplatform.googleapis.com"
}
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

parent = f"projects/{project_id}/locations/{region}"

# Define a function to get embeddings
def get_embeddings(texts):
    responses = []
    for text in texts:
        # Construct the payload
        payload = {
            "text_snippet": {"content": text, "mime_type": "text/plain"}
        }
        # Make the embedding request
        response = client.predict(
            name=f"projects/{project_id}/locations/{region}/models/MODEL_ID",
            payload=payload
        )
        responses.append(response)
    
    return responses

# Your code to handle the responses and update the database would go here


In [51]:
def handle_vertex_ai_response(texts):
    # Initialize the Vertex AI client for the prediction service
    client = aiplatform.gapic.PredictionServiceClient(client_options={"api_endpoint": f"{region}-aiplatform.googleapis.com"})
    
    # Assume you have a model that can generate embeddings
    model_name = f"projects/{project_id}/locations/{region}/models/MODEL_ID"

    # Prepare a list for the responses
    embeddings = []
    for text in texts:
        # Create the prediction request
        response = client.predict(
            name=model_name,
            payload={"text_snippet": {"content": text, "mime_type": "text/plain"}}
        )
        # Extract the embedding from the response and append to the list
        # This is a simplification, actual response handling would need to align with the response structure
        embeddings.append(response.predictions)

    return embeddings

In [57]:
def vector_search_create_index_endpoint(
    project: str, location: str, display_name: str ) -> None:
    """Create a vector search index endpoint.

    Args:
        project (str): Required. Project ID
        location (str): Required. The region name
        display_name (str): Required. The index endpoint display name
    """
    # Initialize the Vertex AI client
    aiplatform.init(project=project, location=location)

    # Create Index Endpoint
    index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
        display_name=display_name,
        public_endpoint_enabled=True,
        description="Matching Engine Index Endpoint",
    )

    print(index_endpoint.name)


## new try

In [ ]:
# run this
!pip install google-cloud-aiplatform --upgrade --user

In [ ]:
# making sure that protobuf version matches with what we need 
!pip install protobuf==3.20.0

In [1]:
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials

# Path to your service account key file
key_path = '/Users/alessandracerutti/ucl-invoice-assistant/ucl-engineering-invoice-5d941af64425.json'
credentials = Credentials.from_service_account_file(
    key_path,
    scopes=['https://www.googleapis.com/auth/cloud-platform'])

if credentials.expired:
    credentials.refresh(Request())

In [3]:
from template_specification_gcp_postgres import project_id, region, instance_name, current_user, DB_USER, DB_PASS, DB_NAME

IP_ADRESS = '35.232.156.133'
DB_USER = "postgres" 
DB_PASS = "quick-ale"
DB_NAME = "text-extraction"

In [19]:
from google.cloud import aiplatform
import vertexai
import numpy as np
from vertexai.language_models import TextEmbeddingModel

vertexai.init(project=project_id, location=region, credentials = credentials)

embedding_model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@001")

In [21]:
# Prepare your list of texts
input_text_lst_news = [
    "Inception Mind-bending heist inside dreams blurs reality and imagination",
    "The Shawshank Redemption Hope survives in the darkest of prison walls"
]

In [22]:
for input_text in input_text_lst_news:
    response = embedding_model.get_embeddings([input_text])

    # Print the response to see what it contains
    print(response[0])

    # If printing the response doesn't help, try printing the __dict__ attribute
    print(response[0].__dict__)


TextEmbedding(values=[-0.07580184936523438, 0.006770612206310034, 0.04649105295538902, 0.014426245354115963, 0.0434042327105999, -0.03197837248444557, 0.009617216885089874, 0.003914788365364075, -0.04703899100422859, -0.014279136434197426, -0.0060620796866714954, 0.051147159188985825, -6.720898090861738e-05, -0.02313300035893917, -0.027460120618343353, -0.01495133712887764, -0.02274889498949051, -0.05642899498343468, 0.013038323260843754, 0.05466878041625023, -0.11715470999479294, -0.02910725399851799, 0.03181386739015579, -0.019390247762203217, -0.03609166294336319, -0.0759643018245697, 0.012005242519080639, 0.00013766186020802706, -0.006794617511332035, -0.004293686710298061, -0.0034558111801743507, 0.043806806206703186, -0.01756454072892666, -0.032398950308561325, -0.0502358004450798, 0.017922161146998405, 0.03258422389626503, 0.03941325470805168, -0.01588195376098156, -0.0017146599711850286, 0.005427548196166754, -0.057702869176864624, -0.01755714602768421, 0.03569668531417847, -0.

In [23]:
embeddings = []
for input_text in input_text_lst_news:
    # Get the embeddings for each text
    response = embedding_model.get_embeddings([input_text])

    # Try accessing the 'values' attribute directly
    embedding_values = getattr(response[0], 'values', None)
    if embedding_values is not None:
        embeddings.append(embedding_values)
    else:
        print("No 'values' attribute found in the TextEmbedding object.")

# Convert the list of embeddings to a numpy array
embeddings_array = np.array(embeddings)

print("Shape: " + str(embeddings_array.shape))


Shape: (2, 768)


In [24]:
embeddings_array

array([[-0.07580185,  0.00677061,  0.04649105, ...,  0.01763144,
        -0.05770805, -0.01597101],
       [ 0.00891676, -0.03557374,  0.03205594, ...,  0.0557407 ,
        -0.02079345, -0.03088961]])

### Getting the Data

In [14]:
import psycopg2
from google.cloud.sql.connector import Connector # pip install "cloud-sql-python-connector[pg8000]"
import sqlalchemy # pip install sqlalchemy
from template_specification_gcp_postgres import project_id, region, instance_name, current_user, DB_USER, DB_PASS, DB_NAME

IP_ADRESS = '35.232.156.133'
DB_USER = "postgres" 
DB_PASS = "quick-ale"
DB_NAME = "text-extraction"

# Define the instance connection name
INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" 
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")

# Initialize Connector object
connector = Connector()

# Function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# Create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

import pandas as pd
from sqlalchemy import text

# Function to fetch data from the database
def fetch_data():
    with pool.connect() as conn:
        # Query to select relevant columns
        query = text("SELECT unique_id, extracted_text FROM invoices")
        result = conn.execute(query)
        # Create a DataFrame from the query results
        df = pd.DataFrame(result.fetchall(), columns=["unique_id", "extracted_text"])
        return df

# Fetch the data
df = fetch_data()

# Display the first few rows of the DataFrame to confirm successful data retrieval
print(df.head())

Your instance connection name is: ucl-engineering-invoice:us-central1:quickstart-user
   unique_id                                     extracted_text
0          1  Restaurant du\nCLUB NAUTIQUE MORGIFN\n\nPI. de...
1          2  Verein\nBerufliche Integration CHANCE Z!\nNikl...
2          3  HOTEL SALUVER AG\nVia Maistra 128\n7505 Celeri...
3          4  Fattura\n\n983 360\n\nNumero cliente 7'726\n\n...
4          5  Fattura\n\n983 201\n\nNumero cliente 11'004\n\...


df

In [33]:
from google.cloud import aiplatform
import vertexai
import numpy as np
from vertexai.language_models import TextEmbeddingModel

vertexai.init(project=project_id, location=region, credentials = credentials)

embedding_model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@003")

In [34]:
import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n"],
    chunk_size=500,
    chunk_overlap=80,
    length_function=len
)

In [35]:
# Function to generate embeddings for a list of texts
def generate_embeddings(texts):
    embeddings = []
    for text in texts:
        response = embedding_model.get_embeddings([text])
        embedding_values = getattr(response[0], 'values', None)
        if embedding_values is not None:
            embeddings.append(embedding_values)
        else:
            embeddings.append(None)  # Handle failed embeddings appropriately
    return embeddings

In [37]:
first_row = df.iloc[0]

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n"],
    chunk_size=500,
    chunk_overlap=80,
    length_function=len
)

# Split the text into chunks for the first entry
invoice_id = first_row["unique_id"]
text = first_row["extracted_text"]
splits = text_splitter.create_documents([text])
chunks = [s.page_content for s in splits]

# Generate embeddings for the chunks of the first entry
chunk_embeddings = generate_embeddings(chunks)

In [39]:
chunk_embeddings

[[-0.006655760575085878,
  -0.05844341218471527,
  -0.06520308554172516,
  0.01604229398071766,
  0.07929741591215134,
  0.006215380970388651,
  -0.0067459591664373875,
  -0.03771562874317169,
  -0.003530233632773161,
  0.058100685477256775,
  0.00749293714761734,
  -0.015345614403486252,
  -0.061580318957567215,
  -0.04856882244348526,
  -0.07699994742870331,
  -0.028905851766467094,
  0.031481314450502396,
  0.042643096297979355,
  -0.03599352017045021,
  -0.04231158271431923,
  -0.028168935328722,
  0.008986220695078373,
  0.021957114338874817,
  -4.958905265084468e-05,
  -0.01839350536465645,
  -0.023503420874476433,
  0.008073467761278152,
  -0.056675881147384644,
  -0.04226910322904587,
  -0.036232251673936844,
  -0.06836389005184174,
  0.07340114563703537,
  -0.04030085727572441,
  0.0037920393515378237,
  -0.024629918858408928,
  -0.011581845581531525,
  -0.0036962900776416063,
  0.0730324536561966,
  -0.02678680047392845,
  0.056601814925670624,
  -0.0182724017649889,
  0.0047

In [41]:
if 'embeddings' not in df.columns:
    df['embeddings'] = None  # Initializes the new column


In [42]:
df

,unique_id,extracted_text,embeddings
0,1,Restaurant du\nCLUB NAUTIQUE MORGIFN\n\nPI. de...,NaN
1,2,Verein\nBerufliche Integration CHANCE Z!\nNikl...,NaN
2,3,HOTEL SALUVER AG\nVia Maistra 128\n7505 Celeri...,NaN
3,4,Fattura\n\n983 360\n\nNumero cliente 7'726\n\n...,NaN
4,5,Fattura\n\n983 201\n\nNumero cliente 11'004\n\...,NaN
5,6,Firma\n\nGVS Landi AG\nVinothek\nGennersbrunne...,NaN
6,7,Firma\nGVS Landi AG\n\nVinothek\nGennersbrunne...,NaN
7,8,Café de\nL'UNION PORTUGAISE ;\n\nPlace du Tunn...,NaN


In [45]:
import json

# Serialize the chunk embeddings to a JSON string
serialized_embeddings = json.dumps(chunk_embeddings)

# Now you can safely assign this string to a cell in the DataFrame
df.at[0, 'embeddings'] = serialized_embeddings

/var/folders/vg/nl_vpfpd3f96h8h8tbd78lw40000gn/T/ipykernel_2110/2429472233.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[[-0.006655760575085878, -0.05844341218471527, -0.06520308554172516, 0.01604229398071766, 0.07929741591215134, 0.006215380970388651, -0.0067459591664373875, -0.03771562874317169, -0.003530233632773161, 0.058100685477256775, 0.00749293714761734, -0.015345614403486252, -0.061580318957567215, -0.04856882244348526, -0.07699994742870331, -0.028905851766467094, 0.031481314450502396, 0.042643096297979355, -0.03599352017045021, -0.04231158271431923, -0.028168935328722, 0.008986220695078373, 0.021957114338874817, -4.958905265084468e-05, -0.01839350536465645, -0.023503420874476433, 0.008073467761278152, -0.056675881147384644, -0.04226910322904587, -0.036232251673936844, -0.06836389005184174, 0.07340114563703537, -0.04030085727572441, 0.0037920393515378237, -0.024629918858408928, -0.0115818

In [46]:
df

,unique_id,extracted_text,embeddings
0,1,Restaurant du\nCLUB NAUTIQUE MORGIFN\n\nPI. de...,"[[-0.006655760575085878, -0.05844341218471527,..."
1,2,Verein\nBerufliche Integration CHANCE Z!\nNikl...,NaN
2,3,HOTEL SALUVER AG\nVia Maistra 128\n7505 Celeri...,NaN
3,4,Fattura\n\n983 360\n\nNumero cliente 7'726\n\n...,NaN
4,5,Fattura\n\n983 201\n\nNumero cliente 11'004\n\...,NaN
5,6,Firma\n\nGVS Landi AG\nVinothek\nGennersbrunne...,NaN
6,7,Firma\nGVS Landi AG\n\nVinothek\nGennersbrunne...,NaN
7,8,Café de\nL'UNION PORTUGAISE ;\n\nPlace du Tunn...,NaN


In [36]:
# Split the text into chunks and embed each chunk
embeddings_by_invoice = {}

for index, row in df.iterrows():
    invoice_id = row["unique_id"]
    text = row["extracted_text"]
    
    # Split text into chunks
    splits = text_splitter.create_documents([text])
    chunks = [s.page_content for s in splits]
    
    # Embed chunks and aggregate by invoice_id
    chunk_embeddings = generate_embeddings(chunks)
    
    # You may need to handle multiple chunks per invoice here, e.g., by averaging
    embeddings_by_invoice[invoice_id] = chunk_embeddings

# Now, map the aggregated embeddings back to the DataFrame
df['embeddings'] = df['unique_id'].map(embeddings_by_invoice)

ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: textembedding-gecko. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai.

In [28]:
embeddings_list = generate_embeddings(df['extracted_text'].tolist())

ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: textembedding-gecko. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai.

In [26]:
import numpy as np
from vertexai.language_models import TextEmbeddingModel

# Initialize your embedding model
embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

# Assuming df is your DataFrame and you have a function to generate embeddings

def generate_embeddings_for_text(text):
    response = embedding_model.get_embeddings([text])
    embedding_values = getattr(response[0], 'values', None)
    return embedding_values

# Apply the function to the 'extracted_text' column and create a new column for embeddings
df['extracted_texts_embeddings'] = df['extracted_text'].apply(generate_embeddings_for_text)

# Now df has a new column 'extracted_texts_embeddings' with the embeddings


ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: textembedding-gecko. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai.

In [7]:


# Retrieve embeddings
embeddings = []
for input_text in input_text_lst_news:
    # Get embeddings for each text
    response = embedding_model.get_embeddings([input_text])
    
    # Extract the embedding values assuming the response is structured as expected
    # This part may need adjustment based on the actual response structure
    embedding_values = response[0].embedding

    # Add the extracted values to the embeddings list
    embeddings.append(embedding_values)

# Convert the list of embeddings to a numpy array
embeddings_array = np.array(embeddings)

print("Shape: " + str(embeddings_array.shape))

AttributeError: 'TextEmbedding' object has no attribute 'embedding'

In [59]:
region

'us-central1'

In [56]:
# Create an endpoint
index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"langchain-index-endpoint", public_endpoint_enabled=True
)
if index_endpoint:
    print(f"Index endpoint resource name: {index_endpoint.name}")
    print(
        f"Index endpoint public domain name: {index_endpoint.public_endpoint_domain_name}"
    )

PermissionDenied: 403 Permission denied on resource project your_project_id. [links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
, reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "consumer"
  value: "projects/your_project_id"
}
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
]

In [54]:
from google.cloud import aiplatform_v1
client = aiplatform_v1.PredictionServiceClient(client_options={"api_endpoint": f"{region}-aiplatform.googleapis.com"})

# The endpoint is the full resource name of your deployed model
endpoint = f"projects/{project_id}/locations/{region}/endpoints/{endpoint_id}"

NameError: name 'endpoint_id' is not defined